In [169]:
import pandas as pd
from util import *

In [170]:
# 역명을 코드로 변환하기 위한 data
stationNameCode = pd.read_json(PATH_STATION_INFO_REFINE)
stationNameCode["StationName"] = stationNameCode["StationName"].apply(
    lambda _X: _X.replace("·", ".").replace("국제금융센터", "국제금융")
)

stationNameCode.head()

,StationCode,StationName,LineCode
0,95,다대포해수욕장,1
1,96,다대포항,1
2,97,낫개,1
3,98,신장림,1
4,99,장림,1


In [171]:
HJDONG_AREA_GIS = pd.read_csv("../trimmed_data/HJDONG_AREA_GIS.csv")
FLUX_METRIC = pd.read_csv("../trimmed_data/FLUX_METRIC.csv")
(HJDONG_AREA_GIS.head(3), FLUX_METRIC.head(3))

(   LineCode StationName     HJDONG_NAME  HJDONG_AREA  NEAR_STATION_AREA
 0         1          동매  부산광역시 사하구 괴정1동  1138082.148          36465.620
 1         1          동매  부산광역시 사하구 괴정4동   866771.804         158372.769
 2         1          동매   부산광역시 사하구 당리동  3927869.185          96049.312,
   HJDONG     직장인구    주거인구      유동인구
 0   대저1동  13999.0  5912.0  142306.0
 1   대저2동  22963.0  6615.0  138681.0
 2    강동동   3346.0  3892.0   60696.0)

In [172]:
# HJDONG_NAME에서 '부산광역시 사하구 '를 제거한 새로운 열 생성
# 구군별 csv 파일을 위해 기장군 -> 남구 -> 해운대구... 로 변환
HJDONG_AREA_GIS["CLEANED_HJDONG_NAME"] = HJDONG_AREA_GIS["HJDONG_NAME"].apply(
    lambda _X: _X.split(" ")[-1]
)

# 데이터프레임 결합 및 필요없는 것 drop
df_merged = pd.merge(
    HJDONG_AREA_GIS,
    FLUX_METRIC,
    left_on="CLEANED_HJDONG_NAME",
    right_on="HJDONG",
    how="left",
)
df_merged = df_merged.drop(columns=["CLEANED_HJDONG_NAME", "HJDONG_NAME"])

df_merged
# merged_data.to_csv("잠재수요_기장군.csv", index = False)

,LineCode,StationName,HJDONG_AREA,NEAR_STATION_AREA,HJDONG,직장인구,주거인구,유동인구
0,1,동매,1.138082e+06,36465.620,괴정1동,6612.0,18100.0,60137.0
1,1,동매,8.667718e+05,158372.769,괴정4동,3610.0,15174.0,57599.0
2,1,동매,3.927869e+06,96049.312,당리동,11580.0,24280.0,129003.0
3,1,동매,8.258340e+06,4562.966,하단1동,10245.0,21361.0,206771.0
4,1,동매,1.765292e+06,1765292.312,신평1동,14089.0,15714.0,131105.0
...,...,...,...,...,...,...,...,...
737,4,윗반송,5.475774e+07,490498.920,철마면,4330.0,7641.0,52958.0
738,4,고촌,4.120669e+06,1416792.865,반송2동,6211.0,21487.0,28454.0
739,4,고촌,5.475774e+07,1724737.924,철마면,4330.0,7641.0,52958.0
740,4,안평,4.120669e+06,53355.923,반송2동,6211.0,21487.0,28454.0


In [173]:
# Replacement Dictionary
stationNameCode["sNameCode"] = stationNameCode["StationName"] + stationNameCode[
    "LineCode"
].apply(str)
stationNameCode = stationNameCode.drop(columns=["StationName", "LineCode"])
replace_list = {row.sNameCode: row.StationCode for row in stationNameCode.itertuples()}
# Add Station Code
df_merged["sNameCode"] = df_merged["StationName"] + df_merged["LineCode"].apply(str)
df_merged = df_merged.replace({"sNameCode": replace_list})

# HJDONG_AREA_GIS 데이터 비일관성 부산역 -> 부산
df_merged = df_merged.replace({"sNameCode": {"부산1": 113}})

df_merged = df_merged.drop(columns=["LineCode", "StationName"])
df_merged.head()

,HJDONG_AREA,NEAR_STATION_AREA,HJDONG,직장인구,주거인구,유동인구,sNameCode
0,1138082.148,36465.620,괴정1동,6612.0,18100.0,60137.0,100
1,866771.804,158372.769,괴정4동,3610.0,15174.0,57599.0,100
2,3927869.185,96049.312,당리동,11580.0,24280.0,129003.0,100
3,8258339.734,4562.966,하단1동,10245.0,21361.0,206771.0,100
4,1765292.312,1765292.312,신평1동,14089.0,15714.0,131105.0,100


In [174]:
# CALC IF
df_merged["IMPACT_FACTOR"] = df_merged["NEAR_STATION_AREA"] / df_merged["HJDONG_AREA"]

# APPLY IF
df_merged["직장인구"] *= df_merged["IMPACT_FACTOR"]
df_merged["주거인구"] *= df_merged["IMPACT_FACTOR"]
df_merged["유동인구"] *= df_merged["IMPACT_FACTOR"]

df_merged = df_merged.drop(
    columns=["HJDONG_AREA", "NEAR_STATION_AREA", "HJDONG", "IMPACT_FACTOR"]
)
df_merged.head()

,직장인구,주거인구,유동인구,sNameCode
0,211.857009,579.947347,1926.867049,100
1,659.603477,2772.527193,10524.238421,100
2,283.169062,593.725805,3154.547367,100
3,5.660652,11.802556,114.246819,100
4,14089.000000,15714.000000,131105.000000,100


In [181]:
IMPLICIT_METRIC = df_merged.rename(columns={"sNameCode": "scode"})
IMPLICIT_METRIC = IMPLICIT_METRIC.groupby("scode").sum()
IMPLICIT_METRIC_SUM = IMPLICIT_METRIC.sum(axis=1)

IMPLICIT_METRIC.head()

,직장인구,주거인구,유동인구
scode,,,
95,6870.089447,13797.146686,32959.074917
96,9505.118867,21695.453973,60341.565562
97,11874.448563,28649.458517,114395.518073
98,18045.780876,29726.498244,169798.859466
99,19645.431979,26595.531388,174187.842106


In [183]:
# IMPLICIT_METRIC CHECKSUM
sCodes = stationNameCode["StationCode"].to_list()
for sC in sCodes:
    # 저 멘트 안뜨면 좋은거임.
    assert sC in IMPLICIT_METRIC.index, f"{sC}에서 몬가 이상함."

In [184]:
SAVE_PATH = "../database"
IMPLICIT_METRIC.to_csv(f"{SAVE_PATH}/IMPLICIT_METRIC.csv", encoding="UTF-8")